In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import load_model
import warnings
import pickle
import os
from datetime import datetime
import json
from openpyxl import workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import LineChart, Reference
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# configuration
SEQUENCE_LENGTH = 12
MODEL_DIR = 'cinnamon_models'
DATA_PATH = 'Cinnamon_Dataset_New_0002.csv'
OUTPUT_DIR = 'forecast_exports'

# Global variables for preprocessors
scaler_features = None
scaler_target = None
label_encoders = {}
model_config = {}

# Create output directory
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created output directory: {OUTPUT_DIR}")

print("📊 Batch Cinnamon Price Forecasting to Excel")
print("=" * 60)

def list_available_models(model_dir=MODEL_DIR):
    """List all available saved models"""
    if not os.path.exists(model_dir):
        print(f"❌ Model directory not found: {model_dir}")
        return []
    
    model_folders = []
    for item in os.listdir(model_dir):
        item_path = os.path.join(model_dir, item)
        if os.path.isdir(item_path):
            # Check if it contains required model files
            required_files = ['lstm_model.keras', 'scalers.pkl', 'label_encoders.pkl', 'model_config.json']
            if all(os.path.exists(os.path.join(item_path, f)) for f in required_files):
                model_folders.append(item)
    
    return sorted(model_folders, reverse=True)

def load_saved_model(model_path):
    """Load a previously saved model and preprocessors"""
    global scaler_features, scaler_target, label_encoders, model_config
    
    print(f"📂 Loading model from: {model_path}")
    
    try:
        # Load the Keras model
        keras_model_path = os.path.join(model_path, "lstm_model.keras")
        model = load_model(keras_model_path)
        print(f"✅ Keras model loaded")
        
        # Load scalers
        scalers_path = os.path.join(model_path, "scalers.pkl")
        with open(scalers_path, 'rb') as f:
            scalers = pickle.load(f)
        scaler_features = scalers['scaler_features']
        scaler_target = scalers['scaler_target']
        print(f"✅ Scalers loaded")
        
        # Load label encoders
        encoders_path = os.path.join(model_path, "label_encoders.pkl")
        with open(encoders_path, 'rb') as f:
            label_encoders = pickle.load(f)
        print(f"✅ Label encoders loaded")
        
        # Load configuration
        config_path = os.path.join(model_path, "model_config.json")
        with open(config_path, 'r') as f:
            model_config = json.load(f)
        
        print(f"🎉 Model successfully loaded!")
        print(f"📊 Performance: MAE={model_config['training_info']['mae']:.2f}, "
              f"RMSE={model_config['training_info']['rmse']:.2f}, "
              f"R²={model_config['training_info']['r2']:.4f}")
        
        return model, model_config
        
    except Exception as e:
        print(f"❌ Error loading model: {str(e)}")
        return None, None
    

def load_and_prepare_data(data_path):
    """Load and prepare the cinnamon price dataset"""
    print(f"📊 Loading data from {data_path}...")
    df = pd.read_csv(data_path)
    print(f"Initial data shape: {df.shape}")

    # Convert Month to datetime
    df['Month'] = pd.to_datetime(df['Month'])

    # Handle missing values in Regional_Price
    missing_before = df['Regional_Price'].isna().sum()
    df.loc[df['Is_Active_Region'] == 0, 'Regional_Price'] = df.loc[df['Is_Active_Region'] == 0, 'National_Price']
    missing_after = df['Regional_Price'].isna().sum()
    print(f"Missing Regional_Price values: {missing_before} -> {missing_after}")

    # Encode categorical variables using loaded encoders
    for col in ['Grade', 'Region']:
        if col in label_encoders:
            df[f'{col}_encoded'] = label_encoders[col].transform(df[col])
        else:
            print(f"⚠️ Warning: No encoder found for {col}")

    # Create additional time-based features
    df['Year'] = df['Month'].dt.year
    df['Month_num'] = df['Month'].dt.month
    df['Quarter'] = df['Month'].dt.quarter

    print("Creating lag and rolling features...")

    # Create lag features for key variables
    df = df.sort_values(['Grade', 'Region', 'Month'])
    lag_columns = ['Regional_Price', 'National_Price', 'Temperature', 'Rainfall']
    for col in lag_columns:
        if col in df.columns:
            for lag in [1, 3, 6, 12]:
                df[f'{col}_lag_{lag}'] = df.groupby(['Grade', 'Region'])[col].shift(lag)

    # Create rolling averages
    for col in ['Regional_Price', 'Temperature', 'Rainfall']:
        if col in df.columns:
            for window in [3, 6, 12]:
                df[f'{col}_rolling_{window}'] = df.groupby(['Grade', 'Region'])[col].transform(
                    lambda x: x.rolling(window).mean()
                )

    print(f"Final data shape after feature engineering: {df.shape}")
    return df

# Define training feature columns
TRAIN_FEATURE_COLS = [
    'Grade_encoded', 'Region_encoded', 'Is_Active_Region',
    'National_Price', 'Seasonal_Impact', 'Local_Production_Volume',
    'Local_Export_Volume', 'Global_Production_Volume', 'Global_Consumption_Volume',
    'Temperature', 'Rainfall', 'Exchange_Rate', 'Inflation_Rate', 'Fuel_Price',
    'Year', 'Month_num', 'Quarter',
    'Regional_Price_lag_1', 'Regional_Price_lag_3', 'Regional_Price_lag_6', 'Regional_Price_lag_12',
    'National_Price_lag_1', 'National_Price_lag_3', 'National_Price_lag_6', 'National_Price_lag_12',
    'Temperature_lag_1', 'Temperature_lag_3', 'Temperature_lag_6', 'Temperature_lag_12',
    'Rainfall_lag_1', 'Rainfall_lag_3', 'Rainfall_lag_6', 'Rainfall_lag_12',
    'Regional_Price_rolling_3', 'Regional_Price_rolling_6', 'Regional_Price_rolling_12',
    'Temperature_rolling_3', 'Temperature_rolling_6', 'Temperature_rolling_12',
    'Rainfall_rolling_3', 'Rainfall_rolling_6', 'Rainfall_rolling_12'
]

def forecast_prices(model, df, grade, region, months_ahead=12):
    """Generate price forecasts for specified grade and region"""
    subset = df[(df['Grade'] == grade) & (df['Region'] == region)].sort_values('Month')
    
    if len(subset) == 0:
        print(f"❌ No data found for {grade} in {region}")
        return None, None
    
    last_row = subset.iloc[-1]
    last_date = last_row['Month']
    last_price = last_row['Regional_Price']

    future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1),
                                 periods=months_ahead, freq='MS')
    
    # Generate future rows with realistic seasonal patterns
    future_rows = []
    for future_date in future_dates:
        row = last_row.copy()
        row['Month'] = future_date
        row['Year'] = future_date.year
        row['Month_num'] = future_date.month
        row['Quarter'] = future_date.quarter
        
        # Add seasonal patterns and random variations
        row['Temperature'] = last_row['Temperature'] + 2 * np.sin(2*np.pi*(future_date.month-1)/12) + np.random.normal(0,0.5)
        row['Rainfall'] = max(0, last_row['Rainfall'] + 20 * np.sin(2*np.pi*(future_date.month-1)/12) + np.random.normal(0,10))
        row['Exchange_Rate'] = last_row['Exchange_Rate'] * (1 + np.random.normal(0.001,0.005))
        row['Inflation_Rate'] = last_row['Inflation_Rate'] + np.random.normal(0,0.1)
        row['Fuel_Price'] = last_row['Fuel_Price'] * (1 + np.random.normal(0.002,0.02))
        future_rows.append(row)

    future_df = pd.DataFrame(future_rows)
    extended_df = pd.concat([subset, future_df], ignore_index=True).sort_values('Month')

    # Recreate lag and rolling features for extended data
    for col in ['Regional_Price','National_Price','Temperature','Rainfall']:
        for lag in [1,3,6,12]:
            extended_df[f'{col}_lag_{lag}'] = extended_df.groupby(['Grade','Region'])[col].shift(lag)
        for window in [3,6,12]:
            extended_df[f'{col}_rolling_{window}'] = extended_df.groupby(['Grade','Region'])[col].transform(
                lambda x: x.rolling(window).mean()
            )

    # Select exactly the features used during training
    feature_cols = [c for c in TRAIN_FEATURE_COLS if c in extended_df.columns]
    missing_cols = [c for c in TRAIN_FEATURE_COLS if c not in extended_df.columns]
    if missing_cols:
        print(f"⚠️ Warning: Missing feature columns: {missing_cols}")

    forecasts = []
    historical_data = extended_df[extended_df['Month'] <= last_date]

    for i in range(months_ahead):
        # Get the sequence needed for prediction
        current_data = extended_df.iloc[len(historical_data)-SEQUENCE_LENGTH+i : len(historical_data)+i]
        
        if len(current_data) < SEQUENCE_LENGTH:
            # Pad with last known data if needed
            padding_needed = SEQUENCE_LENGTH - len(current_data)
            last_known = historical_data.iloc[-1:].copy()
            padding_data = pd.concat([last_known]*padding_needed, ignore_index=True)
            current_data = pd.concat([padding_data, current_data], ignore_index=True).iloc[-SEQUENCE_LENGTH:]

        # Prepare sequence for model
        sequence = current_data[feature_cols].fillna(method='ffill').fillna(method='bfill').values
        sequence_flat = sequence.reshape(-1, sequence.shape[-1])
        sequence_scaled_flat = scaler_features.transform(sequence_flat)
        sequence_scaled = sequence_scaled_flat.reshape(sequence.shape)

        # Make prediction
        next_pred = model.predict(sequence_scaled.reshape(1, SEQUENCE_LENGTH, -1), verbose=0)
        next_pred_unscaled = scaler_target.inverse_transform(next_pred)[0][0]
        forecasts.append(next_pred_unscaled)

        # Update the extended dataframe with the new prediction
        future_idx = len(historical_data)+i
        extended_df.iloc[future_idx, extended_df.columns.get_loc('Regional_Price')] = next_pred_unscaled
        extended_df.iloc[future_idx, extended_df.columns.get_loc('National_Price')] = next_pred_unscaled

    return forecasts, future_dates, last_price

def generate_batch_forecasts(model, df, region, months_ahead=12):
    """Generate forecasts for all grades in a specific region"""
    
    # Get available grades for this region
    available_grades = sorted(df[df['Region'] == region]['Grade'].unique())
    
    if not available_grades:
        print(f"❌ No grades found for region: {region}")
        return None
    
    print(f"🔮 Generating forecasts for {len(available_grades)} grades in {region}...")
    
    batch_results = {}
    
    for i, grade in enumerate(available_grades, 1):
        print(f"   📊 Processing {grade} ({i}/{len(available_grades)})...")
        
        try:
            forecasts, future_dates, last_price = forecast_prices(model, df, grade, region, months_ahead)
            
            if forecasts is not None:
                # Calculate additional metrics
                avg_forecast = np.mean(forecasts)
                min_forecast = np.min(forecasts)
                max_forecast = np.max(forecasts)
                std_forecast = np.std(forecasts)
                volatility = (std_forecast / avg_forecast) * 100
                
                # Calculate trend
                trend_pct = ((forecasts[-1] - forecasts[0]) / forecasts[0]) * 100 if len(forecasts) > 1 else 0
                trend_direction = "Increasing" if trend_pct > 0 else "Decreasing" if trend_pct < 0 else "Stable"
                
                # Risk assessment
                if volatility > 15:
                    risk_level = "High"
                elif volatility > 8:
                    risk_level = "Medium"
                else:
                    risk_level = "Low"
                
                batch_results[grade] = {
                    'forecasts': forecasts,
                    'future_dates': future_dates,
                    'last_price': last_price,
                    'avg_forecast': avg_forecast,
                    'min_forecast': min_forecast,
                    'max_forecast': max_forecast,
                    'volatility': volatility,
                    'trend_pct': trend_pct,
                    'trend_direction': trend_direction,
                    'risk_level': risk_level
                }
                print(f"      ✅ Success - Avg: LKR {avg_forecast:,.0f}, Trend: {trend_pct:+.1f}%")
            else:
                print(f"      ❌ Failed to generate forecast")
                
        except Exception as e:
            print(f"      ❌ Error: {str(e)}")
    
    print(f"✅ Batch forecast completed! {len(batch_results)}/{len(available_grades)} successful.")
    return batch_results

def create_excel_report(batch_results, region, months_ahead, output_dir=OUTPUT_DIR):
    """Create a comprehensive Excel report with multiple sheets"""
    
    if not batch_results:
        print("❌ No forecast results to export!")
        return None
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Cinnamon_Forecast_{region}_{months_ahead}months_{timestamp}.xlsx"
    filepath = os.path.join(output_dir, filename)
    
    print(f"📝 Creating Excel report: {filename}")
    
    # Create workbook with multiple sheets
    wb = Workbook()
    
    # Remove default sheet
    wb.remove(wb.active)
    
    # Define styling
    header_font = Font(bold=True, color='FFFFFF')
    header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    subheader_font = Font(bold=True, color='000000')
    subheader_fill = PatternFill(start_color='D6EAF8', end_color='D6EAF8', fill_type='solid')
    border = Border(left=Side(style='thin'), right=Side(style='thin'), 
                   top=Side(style='thin'), bottom=Side(style='thin'))
    center_align = Alignment(horizontal='center', vertical='center')
    
    # Sheet 1: Summary Dashboard
    ws_summary = wb.create_sheet("Summary Dashboard")
    
    # Summary header
    ws_summary['A1'] = f"🌿 Cinnamon Price Forecast Summary - {region}"
    ws_summary['A1'].font = Font(bold=True, size=16)
    ws_summary['A2'] = f"📅 Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    ws_summary['A3'] = f"📊 Forecast Period: {months_ahead} months"
    ws_summary['A4'] = f"🏆 Model R²: {model_config.get('training_info', {}).get('r2', 'N/A'):.4f}"
    
    # Summary table headers
    headers = ['Grade', 'Last Price (LKR)', 'Avg Forecast (LKR)', 'Min Forecast (LKR)', 
               'Max Forecast (LKR)', 'Volatility (%)', 'Trend (%)', 'Trend Direction', 'Risk Level']
    
    for col, header in enumerate(headers, 1):
        cell = ws_summary.cell(row=6, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    # Fill summary data
    row = 7
    for grade, data in batch_results.items():
        ws_summary[f'A{row}'] = grade
        ws_summary[f'B{row}'] = round(data['last_price'], 2)
        ws_summary[f'C{row}'] = round(data['avg_forecast'], 2)
        ws_summary[f'D{row}'] = round(data['min_forecast'], 2)
        ws_summary[f'E{row}'] = round(data['max_forecast'], 2)
        ws_summary[f'F{row}'] = round(data['volatility'], 2)
        ws_summary[f'G{row}'] = round(data['trend_pct'], 2)
        ws_summary[f'H{row}'] = data['trend_direction']
        ws_summary[f'I{row}'] = data['risk_level']
        
        # Apply styling
        for col in range(1, 10):
            cell = ws_summary.cell(row=row, column=col)
            cell.border = border
            cell.alignment = center_align
            
            # Color code risk levels
            if col == 9:  # Risk Level column
                if data['risk_level'] == 'High':
                    cell.fill = PatternFill(start_color='FFEBEE', end_color='FFEBEE', fill_type='solid')
                elif data['risk_level'] == 'Low':
                    cell.fill = PatternFill(start_color='E8F5E8', end_color='E8F5E8', fill_type='solid')
            
            # Color code trends
            elif col == 7:  # Trend % column
                if data['trend_pct'] > 0:
                    cell.fill = PatternFill(start_color='E8F5E8', end_color='E8F5E8', fill_type='solid')
                elif data['trend_pct'] < 0:
                    cell.fill = PatternFill(start_color='FFEBEE', end_color='FFEBEE', fill_type='solid')
        
        row += 1
    
    # Auto-adjust column widths
    for col in ws_summary.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws_summary.column_dimensions[column].width = adjusted_width
    
    # Sheet 2: Detailed Forecasts
    ws_detailed = wb.create_sheet("Detailed Forecasts")
    
    # Detailed forecasts header
    ws_detailed['A1'] = f"📊 Detailed Monthly Forecasts - {region}"
    ws_detailed['A1'].font = Font(bold=True, size=14)
    
    # Create detailed forecast table
    current_row = 3
    
    # Headers for detailed table
    detail_headers = ['Month', 'Date'] + list(batch_results.keys())
    for col, header in enumerate(detail_headers, 1):
        cell = ws_detailed.cell(row=current_row, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    current_row += 1
    
    # Get the first grade's dates (all should be the same)
    first_grade = list(batch_results.keys())[0]
    future_dates = batch_results[first_grade]['future_dates']
    
    # Fill monthly forecast data
    for month_idx, date in enumerate(future_dates):
        ws_detailed[f'A{current_row}'] = f"Month {month_idx + 1}"
        ws_detailed[f'B{current_row}'] = date.strftime('%Y-%m')
        
        for col_idx, grade in enumerate(batch_results.keys(), 3):
            forecast_value = batch_results[grade]['forecasts'][month_idx]
            cell = ws_detailed.cell(row=current_row, column=col_idx, value=round(forecast_value, 2))
            cell.border = border
            cell.alignment = center_align
        
        current_row += 1
    
    # Auto-adjust column widths for detailed sheet
    for col in ws_detailed.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 15)
        ws_detailed.column_dimensions[column].width = adjusted_width
    
    # Sheet 3: Analysis & Insights
    ws_analysis = wb.create_sheet("Analysis & Insights")
    
    ws_analysis['A1'] = f"🎯 Market Analysis - {region}"
    ws_analysis['A1'].font = Font(bold=True, size=14)
    
    current_row = 3
    
    # Market insights
    avg_prices = [data['avg_forecast'] for data in batch_results.values()]
    volatilities = [data['volatility'] for data in batch_results.values()]
    trends = [data['trend_pct'] for data in batch_results.values()]
    
    # Find best/worst performers
    best_price_grade = max(batch_results.keys(), key=lambda x: batch_results[x]['avg_forecast'])
    lowest_price_grade = min(batch_results.keys(), key=lambda x: batch_results[x]['avg_forecast'])
    most_volatile_grade = max(batch_results.keys(), key=lambda x: batch_results[x]['volatility'])
    least_volatile_grade = min(batch_results.keys(), key=lambda x: batch_results[x]['volatility'])
    best_trend_grade = max(batch_results.keys(), key=lambda x: batch_results[x]['trend_pct'])
    worst_trend_grade = min(batch_results.keys(), key=lambda x: batch_results[x]['trend_pct'])
    
    insights = [
        ("📊 MARKET OVERVIEW", ""),
        ("Total Grades Analyzed", len(batch_results)),
        ("Average Market Price", f"LKR {np.mean(avg_prices):,.0f}"),
        ("Average Volatility", f"{np.mean(volatilities):.1f}%"),
        ("Average Trend", f"{np.mean(trends):+.1f}%"),
        ("", ""),
        ("🏆 TOP PERFORMERS", ""),
        ("Highest Price Grade", f"{best_price_grade} (LKR {batch_results[best_price_grade]['avg_forecast']:,.0f})"),
        ("Lowest Price Grade", f"{lowest_price_grade} (LKR {batch_results[lowest_price_grade]['avg_forecast']:,.0f})"),
        ("Best Growth Trend", f"{best_trend_grade} ({batch_results[best_trend_grade]['trend_pct']:+.1f}%)"),
        ("Weakest Growth", f"{worst_trend_grade} ({batch_results[worst_trend_grade]['trend_pct']:+.1f}%)"),
        ("Most Stable Grade", f"{least_volatile_grade} ({batch_results[least_volatile_grade]['volatility']:.1f}% volatility)"),
        ("Most Volatile Grade", f"{most_volatile_grade} ({batch_results[most_volatile_grade]['volatility']:.1f}% volatility)"),
        ("", ""),
        ("⚠️ RISK ASSESSMENT", ""),
    ]
    
    # Add risk breakdown
    risk_counts = {'High': 0, 'Medium': 0, 'Low': 0}
    for data in batch_results.values():
        risk_counts[data['risk_level']] += 1
    
    insights.extend([
        ("High Risk Grades", f"{risk_counts['High']} grades"),
        ("Medium Risk Grades", f"{risk_counts['Medium']} grades"),
        ("Low Risk Grades", f"{risk_counts['Low']} grades"),
    ])
    
    # Write insights to sheet
    for insight in insights:
        ws_analysis[f'A{current_row}'] = insight[0]
        ws_analysis[f'B{current_row}'] = insight[1]
        
        if insight[0].startswith(('📊', '🏆', '⚠️')):
            ws_analysis[f'A{current_row}'].font = subheader_font
            ws_analysis[f'A{current_row}'].fill = subheader_fill
        
        current_row += 1
    
    # Auto-adjust column widths for analysis sheet
    ws_analysis.column_dimensions['A'].width = 25
    ws_analysis.column_dimensions['B'].width = 30
    
    # Save the workbook
    try:
        wb.save(filepath)
        print(f"✅ Excel report saved successfully!")
        print(f"📄 File: {filepath}")
        print(f"📊 Sheets created:")
        print(f"   • Summary Dashboard - Overview of all forecasts")
        print(f"   • Detailed Forecasts - Month-by-month predictions")
        print(f"   • Analysis & Insights - Market intelligence")
        
        return filepath
        
    except Exception as e:
        print(f"❌ Error saving Excel file: {str(e)}")
        return None
